In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template

In [3]:
from datasets import load_dataset # type: ignore
import pandas as pd # type: ignore
import numpy as np # type: ignore
from typing import List, Dict # type: ignore
import json # type: ignore
import os # type: ignore
from tqdm import tqdm # type: ignore
import random
from openai import OpenAI # type: ignore

# config.py
from dotenv import load_dotenv # type: ignore

# 加载 .env 文件
load_dotenv()

# 读取
ZetaTechs_api_key = os.getenv('ZETATECHS_API_KEY')
ZetaTechs_api_base = os.getenv('ZETATECHS_API_BASE')

# 初始化OpenAI客户端
client = OpenAI(api_key=ZetaTechs_api_key, base_url=ZetaTechs_api_base)

In [4]:
class BaseDatasetBuilder:
    def __init__(self):
        """初始化基础数据集构建器"""
        self.datasets = {}
        self.processed_data = []
        # 添加多样化的系统提示词
        self.system_prompts = [
            "你是一个有帮助的AI助手。",
            "作为一个AI助手，你会提供准确、有帮助的回答。",
            "你是一个知识渊博的AI助手，善于解释复杂概念。",
            "你是一个友好的AI助手，会用通俗易懂的方式回答问题。",
            "作为AI助手，你会以专业、客观的方式提供帮助。",
            "你是一个可靠的AI助手，始终保持耐心和专注。",
            "作为AI助手，你擅长提供清晰、结构化的解答。",
            "你是一个认真负责的AI助手，会仔细理解并回答问题。",
            "作为AI助手，你会以严谨的态度提供准确的信息。",
            "你是一个智能AI助手，能够理解上下文并给出恰当的回应。"
            "作为一个AI助理，我会专业、友善地回答你的问题。",
            "我是一个知识渊博的AI，很高兴能帮助你解决问题。",
            "让我们一起探讨这个问题，我会尽我所能提供帮助。",
            "我是一个智能助手，擅长理解和解答各类问题。",
            "作为你的AI伙伴，我会用清晰简洁的方式回答问题。",
            "我是一个全能型AI助手，可以处理各种类型的任务。",
            "作为AI助手，我会以严谨专业的态度回应你的需求。",
            "我是一个AI助理，会用通俗易懂的方式解释复杂概念。",
            "作为你的AI搭档，我会用准确和有见地的方式回答问题。"
        ]

    def _generate_dynamic_prompt(self, user_content: str, model_content: str) -> str:
        """使用GPT生成动态系统提示词"""
        try:
            messages = [
                {"role": "system", "content": "你是一个系统提示词生成器。根据给定的用户问题和AI回答，生成一个合适的系统提示词。提示词应该简洁、专业，并且能够指导AI更好地回答类似的问题。直接返回提示词内容，不要包含任何解释或额外的文字。"},
                {"role": "user", "content": f"用户问题：{user_content}\n\n问题回答：{model_content}\n\n请根据上述对话生成一个合适的系统提示词。"}
            ]
            
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"生成动态提示词时出错: {e}")
            # 如果API调用失败，返回一个固定的提示词
            return np.random.choice(self.system_prompts)
        
    def _get_random_system_prompt(self, user_content: str = "", model_content: str = "") -> str:
        """根据概率选择系统提示词的生成方式"""
        rand = np.random.random()
        
        if rand < 0.40:  # 40%概率返回空提示词
            return ""
        elif rand < 0.995:  # 59.5%概率使用固定提示词
            return np.random.choice(self.system_prompts)
        else:  # 0.5%概率使用GPT生成动态提示词 # 注意，每10%的概率就是1万条数据
            return self._generate_dynamic_prompt(user_content, model_content)
        
    def load_base_datasets(self):
        """加载基础阶段所需的数据集"""
        print("正在加载基础数据集...")
        
        try:
            # 通用指令数据集 - 啥样的都有
            self.datasets['belle'] = load_dataset("BelleGroup/train_1M_CN", trust_remote_code=True) # 917,424 rows
            print("✓ Belle数据集加载完成") # https://huggingface.co/datasets/BelleGroup/train_1M_CN
            
            # 中文维基百科 - 我认为要使用此数据集,还需要进行一些额外的处理,参考:基础阶段_wiki.ipynb, 基础阶段_wiki_异步.ipynb
            # 这里我们暂不使用该数据集
            # self.datasets['wiki'] = load_dataset('wikimedia/wikipedia', '20231101.zh') # 1,380,000 rows
            # print("✓ 维基百科数据集加载完成") # https://huggingface.co/datasets/wikimedia/wikipedia
            
            # Alpaca中文数据集 - 也都是啥都有,相比Belle, 此数据集的input_zh可能不是空的
            self.datasets['alpaca'] = load_dataset('silk-road/alpaca-data-gpt4-chinese') # 52,000 rows
            print("✓ Alpaca中文数据集加载完成") # https://huggingface.co/datasets/silk-road/alpaca-data-gpt4-chinese
            
        except Exception as e:
            print(f"加载数据集时出错: {e}")
            raise
    
    def preprocess_data(self, max_samples_per_dataset=50000):
        """预处理各个数据集"""
        for dataset_name, dataset in self.datasets.items():
            print(f"处理 {dataset_name} 数据集...")
            
            if dataset_name == 'belle':
                processed = self._process_belle(dataset, max_samples_per_dataset)
            # elif dataset_name == 'wiki':
            #     processed = self._process_wiki(dataset, max_samples_per_dataset)
            elif dataset_name == 'alpaca':
                processed = self._process_alpaca(dataset, max_samples_per_dataset)
                
            self.processed_data.extend(processed)
            print(f"✓ {dataset_name} 处理完成，添加 {len(processed)} 条数据")
    
    def _process_belle(self, dataset, max_samples):
        """处理Belle数据集"""
        samples = dataset["train"][:max_samples]
        processed_data = []

        # print(type(dataset), type(samples)) # <class 'datasets.dataset_dict.DatasetDict'> <class 'dict'>
        # print(samples.keys()) # dict_keys(['instruction', 'input', 'output'])
        # print(samples["instruction"][0], type(samples["instruction"])) #  <class 'list'>

        # # 打印第一条数据的结构
        # print("Belle数据集结构示例：")
        # print(samples[0])
        # print("\n数据类型：", type(samples[0]))

        # # 获取数据集的列名
        # print("\n数据集列名：")
        # print(samples.features)

        for i in range(len(samples['instruction'])):
            messages = []
            # 传入用户问题和模型回答来生成系统提示词
            system_prompt = self._get_random_system_prompt(
                user_content=samples['instruction'][i],
                model_content=samples['output'][i]
            )
            
            if system_prompt:
                messages.append({"role": "system", "content": system_prompt})
                
            messages.extend([
                {"role": "user", "content": samples['instruction'][i]},
                {"role": "model", "content": samples['output'][i]}
            ])
            
            processed_data.append({'messages': messages})
            
            # 添加简单的进度显示
            if (i + 1) % 100 == 0:
                print(f"Belle数据集处理进度: {i + 1}/{len(samples)}")
        
        return processed_data
    
    # def _process_wiki(self, dataset, max_samples): # 这个数据集的处理有点意思
    #     """处理维基百科数据集"""
    #     samples = dataset["train"][:max_samples]
    #     processed = []
        
    #     for i in range(len(samples['text'])):
    #         text = samples['text'][i]
    #         if len(text) < 100:  # 过滤过短的文本
    #             continue
                
    #         processed.append({
    #             'type': 'knowledge',
    #             'instruction': '请解释以下内容：',
    #             'input': text[:500],  # 取前500字符作为输入
    #             'output': text[500:1000] if len(text) > 500 else ''  # 后续内容作为输出
    #         })
        
    #     return processed
    
    def _process_alpaca(self, dataset, max_samples):
        """处理Alpaca数据集"""
        samples = dataset["train"][:max_samples]
        processed_data = []

        for i in range(len(samples['instruction_zh'])):
            messages = []
            user_content = (f"{samples['instruction_zh'][i]}\n\n"
                          f"输入：{samples['input_zh'][i]}" if samples['input_zh'][i] 
                          else samples['instruction_zh'][i])
            
            # 传入用户问题和模型回答来生成系统提示词
            system_prompt = self._get_random_system_prompt(
                user_content=user_content,
                model_content=samples['output_zh'][i]
            )
            
            if system_prompt:
                messages.append({"role": "system", "content": system_prompt})
                
            messages.extend([
                {"role": "user", "content": user_content},
                {"role": "model", "content": samples['output_zh'][i]}
            ])
            
            processed_data.append({'messages': messages})
            
            # 添加简单的进度显示
            if (i + 1) % 100 == 0:
                print(f"Alpaca数据集处理进度: {i + 1}/{len(samples)}")
        
        return processed_data
    
    def format_for_training(self):
        """将数据格式化为训练所需的格式"""
        formatted_data = []
        
        for item in self.processed_data:
            dialogue_str = apply_chat_template(item['messages'])
            formatted_data.append({
                'text': dialogue_str,
                'type': 'instruction'  # 保留type字段以便统计
            })
            
        return formatted_data
    
    def save_dataset(self, formatted_data, output_path='./base_stage_data', train_ratio=0.8):
        """保存处理后的数据集"""
        os.makedirs(output_path, exist_ok=True)
        
        # 随机打乱数据
        np.random.shuffle(formatted_data)
        
        # 划分训练集和验证集
        split_idx = int(len(formatted_data) * train_ratio)
        train_data = formatted_data[:split_idx]
        valid_data = formatted_data[split_idx:]
        
        # 保存训练集
        train_path = os.path.join(output_path, 'train.json')
        with open(train_path, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, ensure_ascii=False, indent=2)
            
        # 保存验证集
        valid_path = os.path.join(output_path, 'valid.json')
        with open(valid_path, 'w', encoding='utf-8') as f:
            json.dump(valid_data, f, ensure_ascii=False, indent=2)
            
        print(f"数据集已保存：")
        print(f"- 训练集: {train_path} ({len(train_data)} 条数据)")
        print(f"- 验证集: {valid_path} ({len(valid_data)} 条数据)")
        
        # 输出数据集统计信息
        self._print_dataset_stats(formatted_data)
    
    def _print_dataset_stats(self, data):
        """打印数据集统计信息"""
        print("\n数据集统计信息:")
        
        # 统计不同类型的数据数量
        type_counts = {}
        for item in data:
            type_counts[item['type']] = type_counts.get(item['type'], 0) + 1
            
        print("\n数据类型分布:")
        for data_type, count in type_counts.items():
            percentage = (count / len(data)) * 100
            print(f"- {data_type}: {count} ({percentage:.2f}%)")

def main():
    # 1. 初始化数据集构建器
    builder = BaseDatasetBuilder()
    
    # try:
    # 2. 加载数据集
    builder.load_base_datasets()
        
    # 3. 处理数据集
    builder.preprocess_data(max_samples_per_dataset=50000)
        
    # 4. 格式化数据
    formatted_data = builder.format_for_training()     
        
    # 5. 保存数据集
    builder.save_dataset(formatted_data)
        
    print("\n基础阶段数据集构建完成！")
        
    # except Exception as e:
    #     print(f"构建数据集时出错: {e}")

In [5]:
if __name__ == "__main__":
    main()

正在加载基础数据集...
✓ Belle数据集加载完成
✓ Alpaca中文数据集加载完成
处理 belle 数据集...
Belle数据集处理进度: 100/3
Belle数据集处理进度: 200/3
Belle数据集处理进度: 300/3
Belle数据集处理进度: 400/3
Belle数据集处理进度: 500/3
Belle数据集处理进度: 600/3
Belle数据集处理进度: 700/3
Belle数据集处理进度: 800/3
Belle数据集处理进度: 900/3
Belle数据集处理进度: 1000/3
Belle数据集处理进度: 1100/3
Belle数据集处理进度: 1200/3
Belle数据集处理进度: 1300/3
Belle数据集处理进度: 1400/3
Belle数据集处理进度: 1500/3
Belle数据集处理进度: 1600/3
Belle数据集处理进度: 1700/3
Belle数据集处理进度: 1800/3
Belle数据集处理进度: 1900/3
Belle数据集处理进度: 2000/3
Belle数据集处理进度: 2100/3
Belle数据集处理进度: 2200/3
Belle数据集处理进度: 2300/3
Belle数据集处理进度: 2400/3
Belle数据集处理进度: 2500/3
Belle数据集处理进度: 2600/3
Belle数据集处理进度: 2700/3
Belle数据集处理进度: 2800/3
Belle数据集处理进度: 2900/3
Belle数据集处理进度: 3000/3
Belle数据集处理进度: 3100/3
Belle数据集处理进度: 3200/3
Belle数据集处理进度: 3300/3
Belle数据集处理进度: 3400/3
Belle数据集处理进度: 3500/3
Belle数据集处理进度: 3600/3
Belle数据集处理进度: 3700/3
Belle数据集处理进度: 3800/3
Belle数据集处理进度: 3900/3
Belle数据集处理进度: 4000/3
Belle数据集处理进度: 4100/3
Belle数据集处理进度: 4200/3
Belle数据集处理进度: 4300/3
Belle数据集处理进度: 4400/3
Belle数据集处理进度: 4500/3
Be